In [30]:
import numpy as np
from typing import Tuple
import matplotlib.pyplot as plt

def load_data(path: str): 
    print('Importing data...')
    # format file into a list of tweets:
    lines = []
    with open(path, newline='\n') as file:
        lines = file.readlines()
    # take the last column in a line (the tweet) and split it by spaces:
    for i, line in enumerate(lines):
        lines[i] = line.split('|')[2].split(' ')
    vocab = np.unique(np.concatenate(lines).flat)
    # creating data vector X:
    n = np.size(vocab)
    m = np.size(lines, axis=0)
    X = np.zeros((m,n))
    # populating X:
    for i, line in enumerate(lines):
        for word in line:
            j = np.where(vocab==word)[0]
            X[i,j] += 1

    print('X has {} examples and {} features in BOW format...'.format(m,n))
    return X, np.array(vocab)

In [31]:
X, vocab = load_data('data/cnnhealth.txt')

Importing data...


/opt/anaconda3/envs/tf/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3213: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).shape[axis]


X has 4061 examples and 16909 features in BOW format...


In [32]:
np.where(X[0]==1)

(array([ 3175,  4843,  5878,  6741,  6783,  7815,  8507, 10759, 12760,
        12828, 13057, 13716, 13728, 13851, 13909, 15928, 16051, 16271,
        16553, 16754]),)

In [37]:
vocab[np.where(X[0]!=0)]

array(['An', 'Or,', 'abundance', 'can', 'care', 'e-hypochondriacs.',
       'getting', 'http://cnn.it/1L1t1Fv\r\n', 'info', 'into', 'lead',
       'need', 'neglect', 'of', 'online', 'the', 'to', 'turn', 'us', 'we',
       'worse,'], dtype='<U161')

In [39]:
lines = []
with open('data/cnnhealth.txt', newline='\n') as file:
    lines = file.readlines()

In [40]:
lines[0]

'576880531301801984|Sat Mar 14 23:00:11 +0000 2015|An abundance of online info can turn us into e-hypochondriacs. Or, worse, lead us to neglect getting the care we need http://cnn.it/1L1t1Fv\r\n'